In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
from torch.nn.functional import interpolate 
import matplotlib.pyplot as plt
import pydub
from pydub import AudioSegment
from pathlib import Path
import time
# generation imports
from pippi.soundbuffer import SoundBuffer
from pippi import dsp,fx
import param_generation as pg
import _pickle as pickle
from IPython.display import Audio
from feature_extraction.mir_utils import *
###
import torch.utils.data as utils
import torchvision.transforms as transforms
from PIL import Image
###
import scipy.stats as ss
import common_vars as comv
import imp
import librosa
import librosa.display
import sys
import csv
from scipy.io.wavfile import write

sys.path.append("..")

from feature_extraction import resnet

import helpers
imp.reload(resnet)
imp.reload(helpers)
imp.reload(comv)
imp.reload(pg)
from feature_extraction import pytorch_models as tm
imp.reload(tm)
from helpers import *

# from common_vars import SR
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

drum_types = ['hat', 'kick', 'shake', 'snare']

# save_path="./generated_sample_pack/"
# device = "cpu"
# stack_size=3
# BATCH_SIZE=1
# NUM_BINS=100

# classes=comv.classes
# classes_ranked=comv.classes_ranked
# cDict={v:i for i,v in enumerate(classes)}

# drum_groups=['tom_low','snare','hihat_closed','rim','synth_noise','clap','kick','hihat_open','tom_mid']


# drum_df=pd.DataFrame(columns=["name","fc","cnnlstm","env+freq","consensus","stack_size"])

In [2]:
# # transformations to the audio (by default, the SR rate of generations is 44100)
# # model is trained on 5512 SR
SCALE_FACTOR = 1/8

# signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape(1,-1)
# Audio(signal.reshape([-1]),rate=5512,autoplay=True)

In [3]:
import torchaudio
spec = torchaudio.functional.spectrogram

FREQ_BINS=30
TIME_STEPS=10
class specTrans(object):
    def __init__(self,num_mels=50,sr=SR,time_steps=20,amp_to_power=True):
        self.sr = sr
        self.amp_to_power=amp_to_power
        self.num_mels=num_mels
        self.ampP=torchaudio.transforms.AmplitudeToDB(stype='power',top_db=60)
        self.melP=torchaudio.transforms.MelScale(n_mels=self.num_mels, sample_rate=sr,n_stft=None)
        self.hop_step=time_steps-1
    def __call__(self, sample):
        
        wf=sample
        wf=wf.reshape(-1,len(wf))
        sample_length=self.sr

        num_bins=wf[0].shape[0]
        win_length=self.sr//17
        hop_step=self.sr//self.hop_step
        window=torch.tensor([1]*win_length)
        s=spec(wf, 0, window, num_bins, hop_step, win_length,2,normalized=True)
        s=self.melP(s)
        if self.amp_to_power:
            s=self.ampP(s)
        s = s - s.min()
        s = s/s.abs().max()

        freq=s
        freq[torch.isnan(freq)]=0
        freq=freq
        return freq.detach()
    
spec_tf = specTrans(FREQ_BINS,time_steps=TIME_STEPS,sr=SR//4,amp_to_power=True)

In [4]:
cnet = resnet.resnet18(in_channel=1, out_channel=2).to(device)
cnet.load_state_dict( torch.load("feature_extraction/models/resnet18/resnet18-2.checkpoint")['model_state_dict'])
cnet.to(device)
cnet.eval()
pass

In [5]:

# cnet = tm.Conv_Spec_DVN(embed_only=False,device=device).to(device)
# checkpoint = torch.load("feature_extraction/models/1d_conv/0.401_0.0237_.checkpoint")
# cnet.load_state_dict(checkpoint['model_state_dict'])
# cnet.to(device)
# cnet.eval()

cnet_dvd = tm.Conv_Spec_DVD(embed_only=False,device=device).to(device)
checkpoint = torch.load("feature_extraction/models/1d_conv_dvd/0.559_0.1256_.checkpoint")
cnet_dvd.load_state_dict(checkpoint['model_state_dict'])
cnet_dvd.to(device)
cnet_dvd.eval()
pass

In [6]:

# make sound and evaluate
def make_and_eval(stack_size = 1):
  
    out,params = StackMaker(stack_size)
    a = memToAud(out)
#     resnet_a = librosa.resample(a,11025,44100)
    try:
        a = helpers.butter_bandpass_filter(a,30,15000,48000,order=2)
    except:
        pass
    # load features for resnet 
    
    try:
        resnet_signal = torch.tensor(librosa.resample(a.reshape(-1),44100,11025)).reshape([1,1,-1]).float()[:,:,0:5012]
    except:
          resnet_signal = torch.tensor(np.zeros([1,1,5012])).float()
    # load features for convolutional models
    signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape(1,-1)
    spec_signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = 0.25,recompute_scale_factor=False).reshape(1,-1)
    spectrogram = spec_tf(spec_signal[0].float())
    
    with torch.no_grad():
        ps = cnet(resnet_signal.to(device))
        ps_dvd = cnet_dvd(signal.to(device),spectrogram.to(device))
    return memToAud(out).reshape(-1),signal,ps.cpu().numpy()[0],torch.sigmoid(ps).cpu().numpy()[0],torch.sigmoid(ps_dvd).cpu().numpy()[0]
a,a_downsampled,ps,pss,ps_dvd = make_and_eval()
print(drum_types[np.argmax(ps_dvd)])

a = a.reshape([-1])

a
Audio(a,rate=SR,autoplay=True)

hat


/home/asalimi/miniconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/home/asalimi/miniconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)


In [28]:
#mantra
len('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
#amen
len('aaaaaaaaaaaaaaaaaaaaaaaaaaaaa')

29

In [8]:
# loop until found
biggest_p = []
best_sounds = {}
for i in range(1000):
    print(i,end="\r")
    a,a_ds,ps,pss,pss_dvd= make_and_eval(1)
    if pss[0] > pss[1]:
        print(drum_types[np.argmax(pss_dvd)])
        print(pss[0],pss[1],i)
        
        a = mix_down(a,drum_types[np.argmax(ps_dvd)])
        break
    biggest_p.append(ps[0])
Audio(a[0:SR//2].reshape([-1]),rate=44100,autoplay=True)

snare
0.7310586 0.5 17


NameError: name 'mix_down' is not defined

In [ ]:
a

In [ ]:
%%time
# getting the best sound out of n tries

# best_sounds = {}
# best_sounds = dict.fromkeys(drum_types)
# for k in drum_types:
#     best_sounds[k] = (0,[0])
def f():
    for i in range(100):
        print(i,end="\r")
        a,a_ds,ps,pss,pss_dvd= make_and_eval(4)
        if pss[0]>pss[1]:
            pass
    #         t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
    #         if best_sounds[t][0]<p:
    #             best_sounds[t] = (p,a)
    return 2
x = f()

In [ ]:
# import numpy
# import IPython.display as ipd
# from IPython.display import clear_output
# import time 
# sr = 22050# sample rate
# T = 2.0# seconds
# t = numpy.linspace(0, T, int(T*sr), endpoint=False)# time variable
# x = 0.5*numpy.sin(2*numpy.pi*440*t)
# ipd.Audio(x, rate=sr)

# for i in range(3):
#     x = 0.1*numpy.sin(i*numpy.pi*440*t)
#     ipd.set_matplotlib_close(close=True)
#     ipd.display(ipd.Audio(x, rate=sr,autoplay=True))
#     time.sleep(1)
#     clear_output()

In [ ]:
# #save kit to file
# from scipy.io.wavfile import write
# from pippi import dsp
# write("feature_extraction/kits/kick.wav",SR,best_sounds["kick"][1])

In [ ]:
# from pydub import AudioSegment

# song = AudioSegment.from_wav("feature_extraction/kits/kick.wav")
# k = dsp.read("feature_extraction/kits/kick.wav")

In [11]:
def mix_down(a,cat):
    a = librosa.util.normalize(a)
    if cat=="kick":
        a = helpers.butter_bandpass_filter(a,30,8000,48000,order=2)
        a = librosa.util.normalize(a)
    if cat == "shake":
        a = helpers.butter_bandpass_filter(a,10000,15000,48000,order=2)
        a = librosa.util.normalize(a)
        a = a * 0.6
    if cat == "hat":
        a = helpers.butter_bandpass_filter(a,8000,15000,48000,order=2)
        a = librosa.util.normalize(a)
        a = a * 0.9
    if cat == "snare":
        a = helpers.butter_bandpass_filter(a,300,12000,48000,order=2)
        a = librosa.util.normalize(a)
        a = a * 0.9
        a = a.reshape([-1])
    return a

In [13]:
import threading
from IPython.display import clear_output
# # playing a loop while drum kit is updated in seperate thread
# drum_kit = {}
# drum_kit = dict.fromkeys(drum_types)
# counts = dict.fromkeys(drum_types)
for k in counts:
    counts[k] = 0
    
bpm = 500
num_beats = 16
def make_beat(drum_kit):
    loop = np.zeros(num_beats*60*SR//bpm ) # init silent loop
    loop = add_to_loop(loop,"kick",0,drum_kit)
#     loop = add_to_loop(loop,"kick",2,drum_kit)
    loop = add_to_loop(loop,"snare",4,drum_kit)
    loop = add_to_loop(loop,"hat",6,drum_kit)
    loop = add_to_loop(loop,"shake",7,drum_kit)
    loop = add_to_loop(loop,"hat",8,drum_kit)
#     loop = add_to_loop(loop,"shake",9,drum_kit)
    loop = add_to_loop(loop,"kick",10,drum_kit)
    loop = add_to_loop(loop,"snare",12,drum_kit)
    return loop

# bpm = 300
# num_beats = 7
# def make_beat(drum_kit):
#     loop = np.zeros(num_beats*60*SR//bpm ) # init silent loop
#     loop = add_to_loop(loop,"kick",0,drum_kit)
#     loop = add_to_loop(loop,"hat",1,drum_kit)
#     loop = add_to_loop(loop,"kick",2,drum_kit)
#     loop = add_to_loop(loop,"snare",4,drum_kit)
#     loop = add_to_loop(loop,"shake",5,drum_kit)
#     return loop

def add_to_loop(loop,drum,beat,drum_kit):
        # given a loop, it puts a drum at beat n
        beat_position = int(SR*beat*60//bpm)
        try:
            drum_sound =  drum_kit[drum][1][0:SR//3]
        except:
            drum_sound = np.zeros(SR)
        start,stop = beat_position,min(beat_position+len(drum_sound),len(loop))
        loop[start:stop] += drum_sound[0:stop-start]
        return loop

def play_loop(loop,sr=SR):
        ipd.set_matplotlib_close(close=True)
        ipd.display(ipd.Audio(loop, rate=sr,autoplay=True))
        loop_length = len(loop)/sr
        time.sleep(loop_length)
        clear_output()


def kit_updater(drum_kit,counts,tries=10,):
    # updates a drum kit for n tries
    loops_made = 0
    for i in range(tries):
        a,a_ds,ps,pss,pss_dvd= make_and_eval(np.random.randint(3))
        if pss[0] > pss[1]:
            t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
#             a = mix_down(a,t)
            drum_kit[t] = (p,a)
            counts[t] += 1
            loop = make_beat(drum_kit)
            write('loops/mantra/loop:%d-attempt:%d.wav'%(loops_made,i), 44100, loop)
            loops_made+=1
        if loops_made>100:
            break

def morph_kit():
    kit_updater_thread = threading.Thread(target=kit_updater,args=(drum_kit,counts,10000))
    kit_updater_thread.start()
    while kit_updater_thread.is_alive():
        loop = make_beat(drum_kit)
        ipd.set_matplotlib_close(close=True)
        ipd.display(ipd.Audio(loop, rate=SR,autoplay=True))
        loop_length = len(loop)/SR
        time.sleep(loop_length)
        clear_output()
# loop = make_beat()
# play_loop(loop)
morph_kit()

In [ ]:
counts

In [ ]:

def kit_updater(drum_kit,tries=10,):
    for i in range(tries):
        a,a_ds,ps,pss,pss_dvd= make_and_eval(1)
        if pss[0]>pss[1]:
            t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
            drum_kit[t] = (p,a)
        
def check_kit():
    kit_updater_thread = threading.Thread(target=kit_updater,args=(drum_kit,500))
    kit_updater_thread.start()
    while kit_updater_thread.is_alive():
        time.sleep(1)
check_kit()

In [ ]:
bpm = 300
num_beats = 7
def make_beat(drum_kit):
    loop = np.zeros(num_beats*60*SR//bpm ) # init silent loop
    loop = add_to_loop(loop,"kick",0,drum_kit)
    loop = add_to_loop(loop,"hat",1,drum_kit)
    loop = add_to_loop(loop,"kick",2,drum_kit)
    loop = add_to_loop(loop,"snare",4,drum_kit)
    loop = add_to_loop(loop,"shake",5,drum_kit)
    return loop
loop = make_beat(best_sounds)
ipd.display(ipd.Audio(loop, rate=SR,autoplay=True))

In [18]:
likes = pd.read_csv("csvs/drum_track_abram.txt")

In [26]:
likes.columns=["loop","bad","c1","c2","c3"]
likes[0:99]["bad"].sum(),likes[99:]["bad"].sum()

(54, 55)